# Chaos during a stampede

Based on the paper: https://www.sciencedirect.com/science/article/pii/S1569190X20300861?casa_token=eoCj844--UoAAAAA%3AW5IEQVZQbP5ILasesMnahd3iIgfHNXfAWnaI7RsboLHrfqNA-UNxb_pyNF5FIyDN0hoKbjdC-EA 

### To do:
Implement everything

### Ideas:
Implement people as class so that we can define their state and position as parts of that class.

Use code example below, set all bordercells to 500 (= wall)
Use different values for different states of people.
Make custom update rules.

### Steps:
1. Populate board with random amount of people in random positions
2. In each time step, calculate the static floor field Sij, the dynamic floor field Dij and the event floor field Fij with help of the formulas.
3. For each pedestrian, determine his/her state and make a decision regarding movement behavior. If the pedestrian is in
state Ue,Un or Ae, calculate the transition probability according to Eqs. (15)–(17), respectively. If the pedestrian is in state An, he/
she chooses one of the strategies S1, S2 or S3 to evacuate. If the pedestrian is in state C, calculate the probability of him/her
getting up again.
4. Each pedestrian moves to his/her target cell. If the pedestrian chooses a target cell that is occupied by a fallen pedestrian, he/she will be tripped and become a new fallen pedestrian.
5. If the chaos duration is over, uninjured pedestrians regard fallen pedestrians as obstacles and conduct a normal evacuation. There will be no more new fallen pedestrians.
6. When multiple pedestrians choose to move to the same target, one pedestrian is randomly selected with equal probability to move to the target cell, and other pedestrians remain in their original cells.
7. Determine whether the evacuation is over. If there are still uninjured pedestrians in the room, return to step 2 and repeat the simulation process until all the uninjured pedestrians evacuate from the room.

### Variables:

S = static floor: describes pedestrian behavior of finding the shortest path to the exit during the evacuation process.
D = dynamic floor: represents the interactions among pedestrians in the evacuation process.
F = event floor: reflects thhat some pedestrians intend to escape from the stampede location when they are affected by the stampede.
A = risk floor field: depicts that pedestrian movement direction affects the risk of being tripped.


In [2]:
# imports

import numpy as np

In [1]:
# function

# step 1
def init_board(r):
    return board 

cellular_stampede = cpl.init_random2d(10,10)

complete = False

# cycle

def calc_S(board):
    return S 

def calc_D(board):
    return D 

def calc_F(board):
    return F 

def calc_tumble(Rper,ka,kc,board):
    #rho = (sum of num of spaces that are occupied)/abs(U0)
    if rho > 0.64: # more then 4 people/m^2 so trample threshold is exceeded
        eps = 1
    else:
        eps = 0
     
    Aij = np.cos(theta) - 1 # theta is de hoek tussen bewegingsrichting van vorige stap en normaal van huidige cell naar cell ij
    if board[i,j] == 500:
        aij = kc*eps*rho*np.exp(ka*Aij)
    else:
        aij = 1
    
    return aij



NameError: name 'cpl' is not defined

In [ ]:
# parameters

kc = 1 # sensitivity parameter for tumble factor 
ka = 1 # sensitivity parameter for tumble factor 
Rper = 1 # perception radius for each person

In [18]:
# main function

while not(complete)

    board = cellular_stampede[-1,:,:]

    # step 2
    
    S = calc_S(board)
    D = calc_D(board)
    F = calc_F(board)

    # step 3 and 4
    # if target cell is occupied state becomes C

    states = {500:'wall', 1:'..'}

    person_pos = board[...]

    next_board = board

    for i in person_pos:
        person_state = states[person_pos]

        if person_state = 'Ue':
            ....
        elif person_state = 'Un':
            ....
        elif person_state = 'Ae':
            ....
        elif person_state = 'An':
            ....
        elif person_state = 'C':
            ....
        else:
            raise ValueError('Non existing person state')

    # step 5

    if T > Tc:
        # C becomes obstacle (= 500)
        # change evacuation strategy of others to normal
        # change to fall = 0

    # step 6
        # make list of steps with 1 current pos en 2 next pos. If multiple person have same next pos than equal chance to stay or move

    # step 7

    if not(uninjured in board):
        complete = True


[[[0 0 0 1 0 0 0 1 1 0]
  [1 0 0 1 0 0 1 0 0 0]
  [0 1 1 1 1 0 0 0 1 0]
  [0 0 1 1 1 1 0 0 0 0]
  [1 0 0 0 0 1 0 0 0 1]
  [1 0 1 0 1 1 1 0 0 0]
  [1 0 0 1 0 0 0 1 1 0]
  [0 1 1 1 1 1 1 1 1 1]
  [0 0 0 0 0 0 0 1 0 1]
  [0 1 1 1 0 1 0 1 1 0]]]
[[0 0 0 1 0 0 0 1 1 0]
 [1 0 0 1 0 0 1 0 0 0]
 [0 1 1 1 1 0 0 0 1 0]
 [0 0 1 1 1 1 0 0 0 0]
 [1 0 0 0 0 1 0 0 0 1]
 [1 0 1 0 1 1 1 0 0 0]
 [1 0 0 1 0 0 0 1 1 0]
 [0 1 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 1 0 1]
 [0 1 1 1 0 1 0 1 1 0]]
